# 🤟 Clasificador de Lengua de Señas Boliviana - Google Colab

Sistema de clasificación de videos de lengua de señas usando Deep Learning, basado en la estrategia de **5 steps**.

## 📊 Dataset

- **1,406 videos** de lengua de señas boliviana
- **71 clases** diferentes
- Videos cortos (~3 segundos promedio)
- Resolución 1280x720

## 🎯 Objetivos

Entrenar **2 modelos** de clasificación:

1. **Lightweight**: Modelo rápido (~150K parámetros) - 2-4 horas entrenamiento
2. **R(2+1)D**: Modelo de alta accuracy (~31M parámetros) - 6-10 horas entrenamiento

---

## 📋 Antes de empezar:

1. **Configura GPU**: Runtime → Change runtime type → **T4 GPU**
2. **Duración**: ~8-14 horas para ambos modelos
3. **Espacio**: ~10-15GB necesarios

---

### ⚙️ Hardware Recomendado:
- **GPU**: T4 (16GB VRAM) ✅ RECOMENDADO
- **RAM**: 12GB High-RAM

**¿Por qué T4 GPU?**
- Modelos 3D CNN optimizados para CUDA
- Mejor rendimiento que CPU para videos
- 16GB VRAM suficiente para ambos modelos

## 🔧 1. Setup Inicial

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"{'='*60}")

## 📦 2. Instalar Dependencias

In [ ]:
# Instalar PyTorch con CUDA (optimizado para Colab T4)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Instalar resto de dependencias
!pip install opencv-python pandas matplotlib seaborn scikit-learn tqdm -q

print("✅ Instalación completada")

## 💾 3. Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Crear directorios en Drive
import os
DRIVE_PATH = '/content/drive/MyDrive/LSB_Classifier'
os.makedirs(f"{DRIVE_PATH}/checkpoints_lightweight", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/checkpoints_r2plus1d", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/logs", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/data", exist_ok=True)

print(f"✅ Google Drive montado en: {DRIVE_PATH}")

## 📁 4. Preparar Datos (usando estrategia 5 steps)

In [ ]:
# Cambiar al directorio del proyecto clonado
%cd step5-lsb/codigo

print("📂 Cambiado a directorio del proyecto")
!pwd
!ls -la

In [ ]:
# Crear enlace simbólico a la carpeta videos en Drive
!ln -sf /content/drive/MyDrive/LSB_TAD/data/videos ../videos

print("✅ Enlace simbólico creado: ../videos -> /content/drive/MyDrive/LSB_TAD/data/videos")

In [ ]:
# Copiar videos localmente para acelerar el entrenamiento
print("🚀 COPIANDO VIDEOS LOCALMENTE PARA ACELERAR ENTRENAMIENTO...")
print("Esto tomará 5-10 minutos pero acelerará 6x el entrenamiento")

import os
import shutil
from pathlib import Path

LOCAL_VIDEOS = Path('/content/videos_local')
DRIVE_VIDEOS = Path('/content/drive/MyDrive/LSB_TAD/data/videos')

# Crear directorio local
LOCAL_VIDEOS.mkdir(exist_ok=True)

# Copiar videos
if DRIVE_VIDEOS.exists():
    print(f"📂 Copiando desde: {DRIVE_VIDEOS}")
    print(f"📂 Hacia: {LOCAL_VIDEOS}")
    
    # Copiar recursivamente
    shutil.copytree(DRIVE_VIDEOS, LOCAL_VIDEOS / 'videos', dirs_exist_ok=True)
    
    # Cambiar enlace simbólico
    videos_link = Path('../videos')
    if videos_link.exists():
        videos_link.unlink()
    videos_link.symlink_to(LOCAL_VIDEOS / 'videos')
    
    print("✅ Videos copiados localmente")
    print(f"🔗 Enlace simbólico actualizado: ../videos -> {LOCAL_VIDEOS / 'videos'}")
else:
    print("❌ No se encontró la carpeta de videos en Drive")
    print(f"   Verifica la ruta: {DRIVE_VIDEOS}")

print("🎯 Ahora el entrenamiento será mucho más rápido!")

In [ ]:
# Paso 1: Analizar el dataset
print("🔍 PASO 1: Analizando el dataset...")
!python step1_analizar_dataset.py

In [ ]:
# Crear enlace simbólico a la carpeta videos en Drive
!ln -sf /content/drive/MyDrive/LSB_TAD/data/videos ../videos

print("✅ Enlace simbólico creado: ../videos -> /content/drive/MyDrive/LSB_TAD/data/videos")

In [ ]:
# Paso 2: Preparar los datos (dividir train/val/test)
print("\n📊 PASO 2: Preparando datos...")
!python step2_preparar_datos.py

In [ ]:
# Paso 3: Crear dataset y probar DataLoader
print("\n🧪 PASO 3: Probando DataLoader...")
!python step3_crear_dataset.py

In [ ]:
# Paso 4: Probar los modelos
print("\n🏗️ PASO 4: Probando modelos...")
!python step4_crear_modelo.py

## 🚀 5. Entrenar Modelo 1: Lightweight (Recomendado para empezar)

### 📊 Modelo Lightweight
- **Parámetros**: ~150K
- **Tiempo estimado**: 2-4 horas
- **Accuracy esperado**: 60-75%
- **Ventajas**: Rápido, poca memoria, ideal para pruebas

**Configuración optimizada para GPU T4:**
- Batch size: 8 (acumulación de gradientes)
- Learning rate: 0.001
- Épocas: 30
- Frames por video: 8
- Resolución: 112x112

In [ ]:
# Entrenar modelo Lightweight
print("🚀 ENTRENANDO MODELO LIGHTWEIGHT...")
print("⏱️ Tiempo estimado: 2-4 horas")
print("💾 Parámetros: ~150K")
print("="*60)

!python step5_entrenar.py \
    --model lightweight \
    --epochs 30 \
    --batch_size 8 \
    --lr 0.001 \
    --patience 5 \
    --num_frames 8 \
    --frame_size 112

print("\n✅ Entrenamiento Lightweight completado!")

## 🚀 6. Entrenar Modelo 2: R(2+1)D (Alta Accuracy)

### 📊 Modelo R(2+1)D
- **Parámetros**: ~31M
- **Tiempo estimado**: 6-10 horas
- **Accuracy esperado**: 75-85%
- **Ventajas**: Mejor accuracy, arquitectura probada

**Configuración optimizada para GPU T4:**
- Batch size: 4 (con acumulación de gradientes)
- Learning rate: 0.001
- Épocas: 20
- Frames por video: 16
- Resolución: 224x224

In [ ]:
# Entrenar modelo R(2+1)D
print("🚀 ENTRENANDO MODELO R(2+1)D...")
print("⏱️ Tiempo estimado: 6-10 horas")
print("💾 Parámetros: ~31M")
print("="*60)

!python step5_entrenar.py \
    --model r2plus1d \
    --epochs 20 \
    --batch_size 4 \
    --lr 0.001 \
    --patience 5 \
    --num_frames 16 \
    --frame_size 224

print("\n✅ Entrenamiento R(2+1)D completado!")

## 📊 7. Evaluar Modelos

In [ ]:
# Evaluar modelo Lightweight
print("📊 EVALUANDO MODELO LIGHTWEIGHT...")
!python step6_evaluar.py --model_path checkpoints_lightweight/best_model.pth

print("\n" + "="*60)

# Evaluar modelo R(2+1)D
print("📊 EVALUANDO MODELO R(2+1)D...")
!python step6_evaluar.py --model_path checkpoints_r2plus1d/best_model.pth

print("\n✅ Evaluación completada!")

## 🔮 8. Hacer Predicciones

In [ ]:
# Hacer predicción con un video de ejemplo
VIDEO_EJEMPLO = "../videos/1/SALUDOS/HOLA.mp4"  # Cambia esta ruta

print("🔮 PREDICCIÓN CON MODELO LIGHTWEIGHT:")
!python step7_predecir.py --model_path checkpoints_lightweight/best_model.pth --video_path "{VIDEO_EJEMPLO}"

print("\n" + "="*60)

print("🔮 PREDICCIÓN CON MODELO R(2+1)D:")
!python step7_predecir.py --model_path checkpoints_r2plus1d/best_model.pth --video_path "{VIDEO_EJEMPLO}"

print("\n✅ Predicciones completadas!")

## 📈 9. Resumen y Próximos Pasos

### 🎯 Modelos Entrenados
1. **Lightweight**: Modelo rápido para pruebas y prototipos
2. **R(2+1)D**: Modelo de alta accuracy para producción

### 📊 Resultados Esperados

| Modelo | Parámetros | Tiempo | Accuracy | Uso |
|--------|------------|--------|----------|-----|
| Lightweight | ~150K | 2-4h | 60-75% | Pruebas rápidas |
| R(2+1)D | ~31M | 6-10h | 75-85% | Producción |

### 🚀 Próximos Pasos
1. **Evaluación detallada**: Comparar métricas de ambos modelos
2. **Optimización**: Ajustar hiperparámetros si es necesario
3. **Despliegue**: Exportar modelo para inferencia en tiempo real
4. **Streaming**: Integrar con sistema de detección temporal

### 💡 Tips para Mejorar Accuracy
- Aumentar número de épocas
- Usar data augmentation
- Probar diferentes learning rates
- Aumentar resolución de frames (si hay memoria)

---

**¡Felicitaciones! Has entrenado 2 modelos de clasificación de lengua de señas usando la estrategia de 5 steps.** 🎉